In [1]:
import requests
import io
from fastai.vision.all import *
from fastai.vision.widgets import *
import ipywidgets as widgets
from IPython.display import display

# URL of the raw .pkl file on GitHub
url = "https://github.com/KyleAndruczk/cpsc171-hw1-fin/raw/main/bears_model.pkl"

try:
    # Download the .pkl file
    response = requests.get(url)
    response.raise_for_status()
    model_data = io.BytesIO(response.content)

    # Load the saved model from the downloaded data
    learn_inf = load_learner(model_data, cpu=True)

    # Create widgets
    btn_upload = widgets.FileUpload(accept='image/*', multiple=False)
    out_pl = widgets.Output()
    lbl_pred = widgets.Label()
    btn_run = widgets.Button(description='Classify')

    # Define classification function
    def on_click_classify(change):
        try:
            if not btn_upload.value:
                lbl_pred.value = 'Please upload an image first.'
                return

            img_file = next(iter(btn_upload.value.values()))
            img = PILImage.create(img_file['content'])
            out_pl.clear_output()
            with out_pl: display(img.to_thumb(128,128))
            pred,pred_idx,probs = learn_inf.predict(img)
            lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
        except Exception as e:
            lbl_pred.value = f'Error during classification: {str(e)}'

    btn_run.on_click(on_click_classify)

    # Display widgets
    display(widgets.VBox([widgets.Label('Select your bear!'),
                          btn_upload, btn_run, out_pl, lbl_pred]))

except requests.exceptions.RequestException as e:
    print(f"Error downloading the model: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

ModuleNotFoundError: No module named 'fastai'